#ICP6

Download all the required libraries.

In [22]:
# import required modules
#TensorFlow provides a collection of workflows to develop and train models using Python
import tensorflow as tf
#NumPy can be used to perform a wide variety of mathematical operations on arrays.
import numpy as np
import os
import time



Import the dataset and print the dataset

In [23]:
import requests
import re
from bs4 import BeautifulSoup
 
urls = ["https://en.wikipedia.org/wiki/Machine_learning","https://en.wikipedia.org/wiki/Deep_learning","https://en.wikipedia.org/wiki/Artificial_intelligence","https://en.wikipedia.org/wiki/Python_(programming_language)","https://en.wikipedia.org/wiki/Natural_language_processing","https://en.wikipedia.org/wiki/Keras","https://en.wikipedia.org/wiki/Big_data","https://en.wikipedia.org/wiki/Recurrent_neural_network","https://en.wikipedia.org/wiki/Convolutional_neural_network","https://en.wikipedia.org/wiki/Computer_vision"]
text_lists=[]
for url in urls: 
  print(url +"\n")
    # get URL
  page = requests.get(url)
    # scrape webpage
  soup = BeautifulSoup(page.content, 'html.parser')
  valid_tags=["div","p"]
  for tag in soup.find_all('p'):
    if tag.name in valid_tags:
      text_lists.append(tag.get_text())


ip_text="".join(text_lists)
ip_text=re.sub("\[[\d]*]","",ip_text)
print(len(ip_text))
print(ip_text[:300])


https://en.wikipedia.org/wiki/Machine_learning

https://en.wikipedia.org/wiki/Deep_learning

https://en.wikipedia.org/wiki/Artificial_intelligence

https://en.wikipedia.org/wiki/Python_(programming_language)

https://en.wikipedia.org/wiki/Natural_language_processing

https://en.wikipedia.org/wiki/Keras

https://en.wikipedia.org/wiki/Big_data

https://en.wikipedia.org/wiki/Recurrent_neural_network

https://en.wikipedia.org/wiki/Convolutional_neural_network

https://en.wikipedia.org/wiki/Computer_vision

308446
Machine learning (ML) is the study of computer algorithms that can improve automatically through experience and by the use of data. It is seen as a part of artificial intelligence. Machine learning algorithms build a model based on sample data, known as training data, in order to make predictions or


Print the unique characters

In [24]:
# The unique characters in the file
vocab = sorted(set(ip_text))
print ('{} unique characters'.format(len(vocab)))
print(vocab)

117 unique characters
['\n', ' ', '!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '\xa0', '°', '×', 'ç', 'é', 'ï', 'ü', 'Č', 'ń', 'σ', '\u200a', '‑', '–', '—', '‖', '’', '“', '”', '\u2061', 'ℓ', '→', '⇒', '−', '∞']


Process the text

Vectorize the text

Before training, we need to map strings to a numerical representation. Create two lookup tables: one mapping characters to numbers, and another for numbers to characters.

In [25]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in ip_text])

The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task we're training the model to perform. The input to the model will be a sequence of characters, and we train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?

Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain seq_length characters from the text.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right.

So break the text into chunks of seq_length+1. For example, say seq_length is 4 and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello".

To do this first use the tf.data.Dataset.from_tensor_slices function to convert the text vector into a stream of character indices.

In [26]:
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(ip_text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
  print(idx2char[i.numpy()])

M
a
c
h
i


The batch method lets us easily convert these individual characters to sequences of the desired size.

In [27]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'Machine learning (ML) is the study of computer algorithms that can improve automatically through expe'
'rience and by the use of data. It is seen as a part of artificial intelligence. Machine learning algo'
'rithms build a model based on sample data, known as training data, in order to make predictions or de'
'cisions without being explicitly programmed to do so. Machine learning algorithms are used in a wide '
'variety of applications, such as in medicine, email filtering, speech recognition, and computer visio'


For each sequence, duplicate and shift it to form the input and target text by using the map method to apply a simple function to each batch:

In [28]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first examples input and target values:

In [29]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Machine learning (ML) is the study of computer algorithms that can improve automatically through exp'
Target data: 'achine learning (ML) is the study of computer algorithms that can improve automatically through expe'


Create training batches

We used tf.data to split the text into manageable sequences. But before feeding this data into the model, we need to shuffle the data and pack it into batches.

In [30]:

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

Build The Model

Use tf.keras.Sequential to define the model. For this simple example three layers are used to define our model:

tf.keras.layers.Embedding: The input layer. A trainable lookup table that will map the numbers of each character to a vector with embedding_dim dimensions;

tf.keras.layers.LSTM: A type of RNN with size units=rnn_units.Since we have huge amount of data its good to use LSTM layer

Used multiple dense layers to reduce loss .

tf.keras.layers.Dense: The output layer, with vocab_size outputs.

In [31]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform',dropout=0.1),
                               
    
    tf.keras.layers.Dense(vocab_size),
    tf.keras.layers.Dense(50),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

Build the model.

In [32]:
model = build_model(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [33]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 117) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is 100 but the model can be run on inputs of any length:

In [34]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           29952     
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 dense_6 (Dense)             (64, None, 117)           119925    
                                                                 
 dense_7 (Dense)             (64, None, 50)            5900      
                                                                 
 dense_8 (Dense)             (64, None, 117)           5967      
                                                                 
Total params: 5,408,720
Trainable params: 5,408,720
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model we need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to sample from this distribution as taking the argmax of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [35]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
#This gives us, at each timestep, a prediction of the next character index:
sampled_indices

array([ 50,  89,  34,  54,  90,  51, 113, 105,  41,  44, 101,  38,  17,
        13, 100,  75,   4,  73,  68,  21, 111,  79,  93,  45,  45,  59,
        27,  68,  40,  22,  89,  79,  35,  47,   0,   5,  89,  76,  81,
        45,  18,  64,  76,  98, 110,  41,  20,  19,   0,  79,  31,  58,
        61,  29,  43,  95, 110,  56,  73,  13,  49, 100,  79,   0,  35,
        93,  79,  56,  53,  67,  82,  11,  27, 110,  40,  16,  84,  21,
         8,  71,   9,  58,  61,  62, 113,  26,  98,  89,  70, 109,  77,
        66, 103,  94,  31,  66,  77, 113,  93,  99])

Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

Attach an optimizer, and a loss function
The standard tf.keras.losses.sparse_categorical_crossentropy loss function works in this case because it is applied across the last dimension of the predictions.

Because our model returns logits, we need to set the from_logits flag.

In [36]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 117)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.7626567


Configure the training procedure using the tf.keras.Model.compile method. We'll use tf.keras.optimizers.Adam with default arguments and the loss function.

In [37]:
model.compile(optimizer='Adam', loss=loss)

Configure checkpoints

Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints are saved during training:

In [38]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

Execute the training

To keep training time reasonable, use 100 epochs to train the model.

In [39]:
EPOCHS=100
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
47/47 [==============================] - 5s 66ms/step - loss: 3.2905
Epoch 2/100
47/47 [==============================] - 4s 67ms/step - loss: 2.6998
Epoch 3/100
47/47 [==============================] - 3s 66ms/step - loss: 2.4063
Epoch 4/100
47/47 [==============================] - 3s 65ms/step - loss: 2.1720
Epoch 5/100
47/47 [==============================] - 4s 67ms/step - loss: 1.9622
Epoch 6/100
47/47 [==============================] - 4s 67ms/step - loss: 1.7723
Epoch 7/100
47/47 [==============================] - 3s 66ms/step - loss: 1.6211
Epoch 8/100
47/47 [==============================] - 3s 67ms/step - loss: 1.5232
Epoch 9/100
47/47 [==============================] - 3s 67ms/step - loss: 1.4199
Epoch 10/100
47/47 [==============================] - 4s 73ms/step - loss: 1.3511
Epoch 11/100
47/47 [==============================] - 4s 72ms/step - loss: 1.2944
Epoch 12/100
47/47 [==============================] - 3s 67ms/step - loss: 1.2397
Epoch 13/100
47/47 [=====

Generate text

Restore the latest checkpoint
To keep this prediction step simple, use a batch size of 1.

Because of the way the RNN state is passed from timestep to timestep, the model only accepts a fixed batch size once built.

To run the model with a different batch_size, we need to rebuild the model and restore the weights from the checkpoint.

In [40]:
tf.train.latest_checkpoint(checkpoint_dir)
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 256)            29952     
                                                                 
 lstm_3 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 dense_9 (Dense)             (1, None, 117)            119925    
                                                                 
 dense_10 (Dense)            (1, None, 50)             5900      
                                                                 
 dense_11 (Dense)            (1, None, 117)            5967      
                                                                 
Total params: 5,408,720
Trainable params: 5,408,720
Non-trainable params: 0
_________________________________________________________________


The prediction loop

The following code block generates the text:

It Starts by choosing a start string, initializing the RNN state and setting the number of characters to generate.

Get the prediction distribution of the next character using the start string and the RNN state.

Then, use a categorical distribution to calculate the index of the predicted character. Use this predicted character as our next input to the model.

The RNN state returned by the model is fed back into the model so that it now has more context, instead than only one character. After predicting the next character, the modified RNN states are again fed back into the model, which is how it learns as it gets more context from the previously predicted characters.

Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [41]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [42]:
print(generate_text(model, start_string=u"machine learning"))

machine learning model in embedded devices removal intelligence (AGI) architectures. These issues may possibly be addressed by deep learning architectures display problematic behavior bases ranging from advanced missiles to UAVs for recon missions or missile guidance. Space exploration is also common. The opinion of experts and industry insiders is mixed, with sizable fractions both concerned a data-set that has been labeled as "normal" and "abnormal" neural network-based approaches may be viewed as a new paradigm distinct from statistical natural language processing (NLP) algorithms through unsupervised learning encidarnstems a subfield of linguistics, computer science, and artificial intelligence corlow is the necessary and sufficient performance or a deep neural network with ReLU activation is strictly larger than the input dimension, there dar hishore classification, prediction, smaller, spon of fiction, are that uses dynamic BPy-sy were level were lizeles $DL. Hata and late variab